# RL coursework, part II (25 pts total)
---

**Name:** Yuan Lu

**SN:** 20114649

---

**Due date:** *April 10th, 2025*

---

Standard UCL policy (including grade deductions) automatically applies for any late submissions.

## How to submit

When you have completed the exercises and everything has finished running, click on 'File' in the menu-bar and then 'Download .ipynb'. This file must be submitted to Moodle named as **`<studentnumber>_RL_part2.ipynb`** before the deadline above, where `<studentnumber>` is your student number.

----
**Reminder of copyrights**

Copyrights protect this code/content and distribution or usages of it (or parts of it) without permission is prohibited. This includes uploading it and usage of it in training in any LLMs systems.

## Context

In this part, we will take a first look at learning algorithms for sequential decision problems.

## Background reading

* Sutton and Barto (2018), Chapters 3 - 6

# The Assignment

### Objectives

You will use Python to implement several reinforcement learning algorithms.

You will then run these algorithms on a few problems, to understand their properties.

## Setup

### Import Useful Libraries

In [ ]:
import matplotlib.collections as mcoll
import matplotlib.pyplot as plt
import numpy as np

### Set options

In [ ]:
np.set_printoptions(precision=3, suppress=1)

### Some grid world

In [ ]:
W = -100  # wall
G = 100  # goal

GRID_LAYOUT = np.array([
  [W, W, W, W, W, W, W, W, W, W, W, W],
  [W, W, 0, W, W, W, W, W, W, 0, W, W],
  [W, 0, 0, 0, 0, 0, 0, 0, 0, G, 0, W],
  [W, 0, 0, 0, W, W, W, W, 0, 0, 0, W],
  [W, 0, 0, 0, W, W, W, W, 0, 0, 0, W],
  [W, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, W],
  [W, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, W],
  [W, W, 0, 0, 0, 0, 0, 0, 0, 0, W, W],
  [W, W, W, W, W, W, W, W, W, W, W, W]
])

class Grid(object):

  def __init__(self, noisy=False):
    # -1: wall
    # 0: empty, episode continues
    # other: number indicates reward, episode will terminate
    self._layout = GRID_LAYOUT
    self._start_state = (2, 2)
    self._state = self._start_state
    self._number_of_states = np.prod(np.shape(self._layout))
    self._noisy = noisy

  @property
  def number_of_states(self):
    return self._number_of_states

  def get_obs(self):
    y, x = self._state
    return y*self._layout.shape[1] + x

  def obs_to_state(self, obs):
    x = obs % self._layout.shape[1]
    y = obs // self._layout.shape[1]
    s = np.copy(grid._layout)
    s[y, x] = 4
    return s

  def step(self, action):
    y, x = self._state

    if action == 0:  # up
      new_state = (y - 1, x)
    elif action == 1:  # right
      new_state = (y, x + 1)
    elif action == 2:  # down
      new_state = (y + 1, x)
    elif action == 3:  # left
      new_state = (y, x - 1)
    else:
      raise ValueError("Invalid action: {} is not 0, 1, 2, or 3.".format(action))

    new_y, new_x = new_state
    reward = self._layout[new_y, new_x]
    if self._layout[new_y, new_x] == W:  # wall
      discount = 0.9
      new_state = (y, x)
    elif self._layout[new_y, new_x] == 0:  # empty cell
      reward = -1.
      discount = 0.9
    else:  # a goal
      discount = 0.
      new_state = self._start_state

    if self._noisy:
      width = self._layout.shape[1]
      reward += 10*np.random.normal(0, width - new_x + new_y)

    self._state = new_state
    return reward, discount, self.get_obs()

  def plot_grid(self):
    plt.figure(figsize=(4, 4))
    plt.imshow(self._layout != W, interpolation="nearest", cmap='pink')
    plt.gca().grid(0)
    plt.xticks([])
    plt.yticks([])
    plt.title("The grid")
    plt.text(2, 2, r"$\mathbf{S}$", ha='center', va='center')
    plt.text(9, 2, r"$\mathbf{G}$", ha='center', va='center')
    h, w = self._layout.shape
    for y in range(h-1):
      plt.plot([-0.5, w-0.5], [y+0.5, y+0.5], '-k', lw=2)
    for x in range(w-1):
      plt.plot([x+0.5, x+0.5], [-0.5, h-0.5], '-k', lw=2)


### Helper functions
(You should not have to change, or even look at, these.  Do run the cell to make sure the functions are loaded though.)

In [ ]:
def run_experiment(env, agent, number_of_steps):
  mean_reward = 0.
  try:
    action = agent.initial_action()
  except AttributeError:
    action = 0
  for _ in range(number_of_steps):
    reward, discount, next_state = env.step(action)
    action = agent.step(reward, discount, next_state)
    mean_reward += reward
  return mean_reward/float(number_of_steps)

map_from_action_to_subplot = lambda a: (2, 6, 8, 4)[a]
map_from_action_to_name = lambda a: ("up", "right", "down", "left")[a]

def plot_values(grid, values, colormap='pink', vmin=0, vmax=10):
  plt.imshow(values - 1000*(grid<0), interpolation="nearest", cmap=colormap, vmin=vmin, vmax=vmax)
  plt.yticks([])
  plt.xticks([])
  plt.colorbar(ticks=[vmin, vmax])

def plot_action_values(grid, action_values, vmin=-5, vmax=5):
  q = action_values
  fig = plt.figure(figsize=(10, 10))
  fig.subplots_adjust(wspace=0.3, hspace=0.3)
  for a in [0, 1, 2, 3]:
    plt.subplot(4, 3, map_from_action_to_subplot(a))
    plot_values(grid, q[..., a], vmin=vmin, vmax=vmax)
    action_name = map_from_action_to_name(a)
    plt.title(r"$q(s, \mathrm{" + action_name + r"})$")

  plt.subplot(4, 3, 5)
  v = np.max(q, axis=-1)
  plot_values(grid, v, colormap='summer', vmin=vmin, vmax=vmax)
  plt.title("$v(s)$")

  # Plot arrows:
  plt.subplot(4, 3, 11)
  plot_values(grid, grid==0, vmax=1)
  for row in range(len(grid)):
    for col in range(len(grid[0])):
      if grid[row][col] == 0:
        argmax_a = np.argmax(q[row, col])
        if argmax_a == 0:
          x = col
          y = row + 0.5
          dx = 0
          dy = -0.8
        if argmax_a == 1:
          x = col - 0.5
          y = row
          dx = 0.8
          dy = 0
        if argmax_a == 2:
          x = col
          y = row - 0.5
          dx = 0
          dy = 0.8
        if argmax_a == 3:
          x = col + 0.5
          y = row
          dx = -0.8
          dy = 0
        plt.arrow(x, y, dx, dy, width=0.02, head_width=0.4, head_length=0.4, length_includes_head=True, fc='k', ec='k')

def plot_rewards(xs, rewards, color):
  mean = np.mean(rewards, axis=0)
  p90 = np.percentile(rewards, 90, axis=0)
  p10 = np.percentile(rewards, 10, axis=0)
  plt.plot(xs, mean, color=color, alpha=0.6)
  plt.fill_between(xs, p90, p10, color=color, alpha=0.3)

def parameter_study(parameter_values, parameter_name, agent_constructor,
                    env_constructor, color,
                    repetitions=10, number_of_steps=int(1e4)):
  mean_rewards = np.zeros((repetitions, len(parameter_values)))
  greedy_rewards = np.zeros((repetitions, len(parameter_values)))
  for rep in range(repetitions):
    for i, p in enumerate(parameter_values):
      env = env_constructor()
      agent = agent_constructor()
      if 'eps' in parameter_name:
        agent.set_epsilon(p)
      elif 'alpha' in parameter_name:
        agent._step_size = p
      else:
        raise NameError("Unknown parameter_name: {}".format(parameter_name))
      mean_rewards[rep, i] = run_experiment(env, agent, number_of_steps)
      agent.set_epsilon(0.)
      agent._step_size = 0.
      greedy_rewards[rep, i] = run_experiment(env, agent, number_of_steps//10)
      del env
      del agent

  plt.subplot(1, 2, 1)
  plot_rewards(parameter_values, mean_rewards, color)
  plt.yticks=([0, 1], [0, 1])
  plt.ylabel("Average reward over first {} steps".format(number_of_steps), size=12)
  plt.xlabel(parameter_name, size=12)

  plt.subplot(1, 2, 2)
  plot_rewards(parameter_values, greedy_rewards, color)
  plt.yticks=([0, 1], [0, 1])
  plt.ylabel("Final rewards, with greedy policy".format(number_of_steps), size=12)
  plt.xlabel(parameter_name, size=12)

def epsilon_greedy(q_values, epsilon):
  if epsilon < np.random.random():
    return np.argmax(q_values)
  else:
    return np.random.randint(np.array(q_values).shape[-1])


def colorline(x, y, z):
  """
  Based on:
  http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
  http://matplotlib.org/examples/pylab_examples/multicolored_line.html
  Plot a colored line with coordinates x and y
  Optionally specify colors in the array z
  Optionally specify a colormap, a norm function and a line width
  """
  segments = make_segments(x, y)
  lc = mcoll.LineCollection(segments, array=z, cmap=plt.get_cmap('copper_r'),
                            norm=plt.Normalize(0.0, 1.0), linewidth=3)

  ax = plt.gca()
  ax.add_collection(lc)
  return lc


def make_segments(x, y):
  """
  Create list of line segments from x and y coordinates, in the correct format
  for LineCollection: an array of the form numlines x (points per line) x 2 (x
  and y) array
  """
  points = np.array([x, y]).T.reshape(-1, 1, 2)
  segments = np.concatenate([points[:-1], points[1:]], axis=1)
  return segments


def plotting_helper_function(_x, _y, title=None, ylabel=None):
  z = np.linspace(0, 0.9, len(_x))**0.7
  colorline(_x, _y, z)
  plt.plot(0, 0, '*', color='#000000', ms=20, alpha=0.7, label='$w^*$')
  plt.plot(1, 1, '.', color='#ee0000', alpha=0.7, ms=20, label='$w_0$')
  min_y, max_y = np.min(_y), np.max(_y)
  min_x, max_x = np.min(_x), np.max(_x)
  min_y, max_y = np.min([0, min_y]), np.max([0, max_y])
  min_x, max_x = np.min([0, min_x]), np.max([0, max_x])
  range_y = max_y - min_y
  range_x = max_x - min_x
  max_range = np.max([range_y, range_x])
  plt.arrow(_x[-3], _y[-3], _x[-1] - _x[-3], _y[-1] - _y[-3], color='k',
            head_width=0.04*max_range, head_length=0.04*max_range,
            head_starts_at_zero=False)
  plt.ylim(min_y - 0.2*range_y, max_y + 0.2*range_y)
  plt.xlim(min_x - 0.2*range_x, max_x + 0.2*range_x)
  ax = plt.gca()
  ax.ticklabel_format(style='plain', useMathText=True)
  plt.legend(loc=2)
  plt.xticks(rotation=12, fontsize=10)
  plt.yticks(rotation=12, fontsize=10)
  plt.locator_params(nbins=3)
  if title is not None:
    plt.title(title, fontsize=20)
  if ylabel is not None:
    plt.ylabel(ylabel, fontsize=20)


# Section A: Tabular RL

In this section, observations will be states in the environment, so the agent state, environment state, and observation will all be the same, and we will use the word `state` interchangably with `observation`.  You will implement agents, which should be in pure Python - so you cannot use JAX/TensorFlow/PyTorch to compute gradients. Using `numpy` is fine.

### A random agent

Below we show a reference implementation of a simple random agent, implemented according to the interface above.

In [ ]:
class Random(object):

  def __init__(self, number_of_actions, number_of_states, initial_state):
    self._number_of_actions = number_of_actions

  def step(self, reward, discount, next_state):
    next_action = np.random.randint(self._number_of_actions)
    return next_action

### The grid

The cell below shows the `Grid` environment that we will use in this section. Here `S` indicates the start state and `G` indicates the goal.  The agent has four possible actions: up, right, down, and left.  Rewards are: `-100` for bumping into a wall, `+100` for reaching the goal, and `-1` otherwise.  The episode ends when the agent reaches the goal, and otherwise continues.  The discount, on continuing steps, is $\gamma = 0.9$.  Feel free to reference the implemetation of the `Grid` above, under the header "a grid world".

In [ ]:
GRID = Grid()
GRID.plot_grid()


## Q1: Implement TD learning **[4 pts]**
Implement an agent that acts randomly, and _on-policy_ estimates state values $v(s)$, using one-step TD learning with step size $\alpha=0.1$.

Use the `__init__` as provided below.  You should implement the `step` function.  We store the initial state in the constructor because you need its value on the first `step` in order to compute the TD error when the first transition has occurred.

Also implement a property `state_values(self)` returning the vector of all state values (one value per state). (A method with the `@property` decorator can be called without the parentheses as if it's a variable, e.g., `agent.state_values` instead of `agent.state_values()`.)

In [ ]:
class RandomTD(object):

  def __init__(self, number_of_states, number_of_actions, initial_state, step_size=0.1):
      self._number_of_states = number_of_states
      self._number_of_actions = number_of_actions
      self._step_size = step_size
      # Initialize all state values to 0.
      self._values = np.zeros(number_of_states)
      # Store the current state; used to update its value upon receiving a transition.
      self._current_state = initial_state

  @property
  def state_values(self):
    return self._values

  def step(self, reward, discount, next_state):
    # One-step TD update for the current state's value:
    # v(s) <- v(s) + alpha * (reward + discount * v(s') - v(s))
    td_target = reward + discount * self._values[next_state]
    td_error = td_target - self._values[self._current_state]
    self._values[self._current_state] += self._step_size * td_error

    # Update the current state to the new state
    self._current_state = next_state

    # Choose a random action (as the policy is random).
    action = np.random.randint(self._number_of_actions)
    return action

### Run the next cell to run the `RandomTD` agent on a grid world.

If everything worked as expected, the plot below will show the estimates state values under the random policy. This includes values for unreachable states --- on the walls and on the goal (we never actually reach the goal --- rather, the episode terminates on the transition to the goal.  The values on the walls and goal are, and will always remain, zero. The plotting code knows about the walls, so they appear black below.

In [ ]:
# Do not modify this cell.
AGENT = RandomTD(GRID._layout.size, 4, GRID.get_obs())
run_experiment(GRID, AGENT, int(1e5))
v = AGENT.state_values
plot_values(GRID_LAYOUT,
            v.reshape(GRID._layout.shape),
            colormap="pink", vmin=-400, vmax=100)

## Q2: Policy iteration **[5 pts]**
We used TD to do policy evaluation for the random policy on this problem.  Consider doing policy improvement, by taking the greedy policy with respect to a one-step look-ahead.  For this, you may assume we have a true model, so for each state and for each action we can look at the value of the resulting state, and would then pick the action with the highest reward plus subsequent state value. In other words, you can assume we can use $q(s, a) = \mathbb{E}[ R_{t+1} + \gamma v(S_{t+1}) \mid S_t = s, A_t = a]$, where $v$ is the value function learned by TD as implemented. Then we consider the policy that picks the action with the highest action value $q(s, a)$. You do **not** have to implement this, just answer the following question.

2.1 **[3pts]** The above amounts to performing an iteration of policy evaluation and policy improvement.  If we repeat this process over and over again, and repeatedly evaluate the greedy policy and then perform an improvement step by picking the greedy policy, would the policy eventually become optimal?  Explain why or why not in at most three sentences.

> Yes, in a finite MDP, repeatedly evaluating the policy and then choosing the greedy policy with respect to its value function (i.e., policy iteration) will converge to the optimal policy.
By the policy improvement theorem, each greedy update produces a policy that is at least as good as the current one and strictly better unless the current policy is already optimal.
Because there are only finitely many possible policies, this improvement process must terminate in a finite number of steps with the optimal policy.

2.2. **[2pts]** What if you were to run a Q-learning agent on this enveromenint, and consider running to converegence, how would the value function for Q_learning compare to the one learnt by the TD agent?

> When run to convergence, Q‑learning learns the unique fixed point of the Bellman **optimality** operator, i.e. the optimal action‑value function $Q^*(s,a)$. Converting this to state values via $v^*(s)=\max_a Q^*(s,a)$ yields the optimal state‑value function, whereas the TD agent described here only performs policy evaluation for a fixed random policy and thus converges to $v_{\pi_{\text{random}}}(s)$, the value of that suboptimal policy. Consequently, for every state $s$,
$$
v^*(s) \;\ge\; v_{\pi_{\text{random}}}(s),
$$
with strict inequality wherever the random policy is suboptimal, reflecting that Q‑learning discovers the best possible value while the TD agent merely estimates the value of its current (random) policy.



# Section B: Off-policy Bellman operators with function approximation

## Q1: Bellman operator for prediction **[5 pts]**

We are going to implement the _expected_ update to the weight for a simple MDP.
In order to be able to implement this update, we need to know the MDP, as well as the way the agent's value depend on the weights and states.  This is defined as follows:

There are two states, $s_1$ and $s_2$.  All rewards are zero, and therefore can be ignored.  Instead of the underlying state, the agent observes state features $\mathbf{x}_1 = \mathbf{x}(s_1)$ and $\mathbf{x}_2 = \mathbf{x}(s_2)$.  These are defined for the two states are $\mathbf{x}_1 = [1, 1]^{\top}$ and $\mathbf{x}_2 = [2, 1]^{\top}$.  In each state, there are two actions, $a$ and $b$.  Action $a$ always transitions to state $s_1$, action $b$ always transitions to state $s_2$.

![MDP](https://hadovanhasselt.files.wordpress.com/2020/02/mdp.png)

---

The agent will make *linear* predicions, such that $v(s) = \mathbf{w}^\top \mathbf{x}(s)$. In the code cell below, you should implement an update that computes the **expected** weight update, given:
* The current weights `w`.
* A target policy $\pi$ that we want to learn about; the actual input, denoted `pi` in the code below, will be a scalar indicating the probability of selecting action `a` in both states: `pi` $= \pi(a|s), \forall s$.
* A behaviour policy $\mu$ that would be used to generate the transitions; the actual input, denoted `mu` in the code below, will be a scalar indicating the probability of selecting action `a` in both states: `mu` $= \mu(a|s), \forall s$.
* A scalar trace parameter $\lambda$ (=`trace_parameter`),
* A scalar discount parameter $\gamma$ (=`discount`).

The expectation should take into account the probabilities of actions in the future, as well as the steady-state (=long-term) probability of being in a state.  The step size of the update should be $\alpha=0.1$.

The expected update should be for a multi-step $\lambda$-return and should be correct for any pair of target and any behaviour policy.  It should be the expectation of performing a (forward view) TD($\lambda$) update in the MDP described above when the _state distribution_ is generated by the behaviour policy and the _returns_ from each state are generated by the target policy. We will use the update you implement to generate plots below.


In [ ]:
# state features (do not change)
x1 = np.array([1., 1.])
x2 = np.array([2., 1.])

def expected_update(w, pi, mu, trace_parameter, discount, lr):
  alpha = lr

  # Compute current value estimates:
  v1 = np.dot(w, x1)
  v2 = np.dot(w, x2)

  # Under the target policy (pi for action a leads to s1, 1-pi for action b leads to s2),
  # the expected value for the next state is:
  v_bar = pi * v1 + (1 - pi) * v2

  # Compute the multi-step λ-return (forward view) in our reward-free MDP:
  G_lambda = discount * (1 - trace_parameter) / (1 - trace_parameter * discount) * v_bar

  # TD errors for each state:
  td_error1 = G_lambda - v1
  td_error2 = G_lambda - v2

  # The behaviour policy gives state s1 with probability mu, and s2 with probability (1-mu).
  expected_weight_update = mu * td_error1 * x1 + (1 - mu) * td_error2 * x2

  return alpha * expected_weight_update
  #return expected_weight_update

## Experiment: run the cell below
The cell below runs an experiment, across different target policies and trace parameters $\lambda$.

The plots below the cell will show how the weights move within the 2-dimensional weight space, starting from $w_0 = [1, 1]^{\top}$ (shown as red dot).  The optimal solution $w_* = [0, 0]^{\top}$ is also shown (as black star).

In [ ]:
def generate_ws(w, pi, mu, l, g):
  """Apply the expected update 1000 times"""
  ws = [w]
  for _ in range(1000):
    w = w + expected_update(w, pi, mu, l, g, lr=0.1)
    ws.append(w)
  return np.array(ws)

mu = 0.5  # behaviour
g = 0.99  # discount

lambdas = np.array([0, 0.8, 0.9, 0.95, 1.])
pis = np.array([0., 0.1, 0.2, 0.5, 1.])

fig = plt.figure(figsize=(22, 17))
fig.subplots_adjust(wspace=0.25, hspace=0.3)

for r, pi in enumerate(pis):
  for c, l in enumerate(lambdas):
    plt.subplot(len(pis), len(lambdas), r*len(lambdas) + c + 1)
    w = np.ones_like(x1)
    ws = generate_ws(w, pi, mu, l, g)
    title = '$\\lambda={:1.3f}$'.format(l) if r == 0 else None
    ylabel = '$\\pi={:1.1f}$'.format(pi) if c == 0 else None
    plotting_helper_function(ws[:, 0], ws[:, 1], title, ylabel)

## Q2: Analyse results (11 pts total)
1. **[1 pts]** How many of the above 25 experiments diverge?
1. **[1 pts]** For which policies $\pi$, is the true value function $v_{\pi}$ representable in the above feature space (spanned by $x_1, x_2$).
1. **[2 pts]** Why are the results asymmetric across different $\pi$?  In particular, explain why the results look different when comparing $\pi = \pi(a | \cdot) = 0$ to $\pi(a | \cdot) = 1$.
1. **[2 pts]** For which combination of $\pi(a)$ and $\lambda$ does the expected update (with uniform random behaviour) converge? Do not limit the answer to the subset of values in the plots above, but to all possible choices of $\lambda \in [0, 1]$ and $\pi(a|s) \in [0, 1]$, but do restrict yourself to state-less policies, as above, for which the action probabilities are equal in the two states: $\pi(a|s_1) = \pi(a|s_2)$.
1. **[1 pts]** Why do all the plots corresponding to full Monte Carlo update look the same (right column)?
1. **[2 pts]** Why do the plots corresponding to full Monte Carlo have the shape they do?
1. **[2 pts]** The plots above are for uniform behavoiur: $\mu(a|s) = \mu(b|s) = 0.5$.  How would the results change (high level, not in terms of precise plots) if the behaviour policy $\mu$ would select action $a$ more often (e.g., $\mu = 0.8$)?  How would the results change if the behaviour would select $a$ less often (e.g., $\mu = 0.2$)?


## Put answers to Q8 in this cell:
1. **7** divergence:
   - (π=0, λ∈{0,0.8,0.9}),
   - (π=0.1, λ∈{0,0.8}),
   - (π=0.2, λ∈{0,0.8}).
---
2. All policies $pi$ yield a true value function $v_\pi(s_1) = 0$and $v_\pi(s_2) = 0$, because the MDP is reward-free and transitions forever. This zero function is trivially in the span of the two feature vectors $\mathbf{x}_1=[1,1]^\top$ and $\mathbf{x}_2=[2,1]^\top$. Setting $\mathbf{w}=\mathbf{0}$ yields $v(s) = 0$ for both states, matching $v_\pi$. Hence, **every** policy’s value function is representable by some weight vector (specifically $\mathbf{w}=\mathbf{0}$).
---
3. The agent’s update depends on how often the target policy transitions to $s_1$ versus $s_2$. Because $\mathbf{x}_1\neq \mathbf{x}_2$ and the magnitude of $\mathbf{x}_2$ is larger in the first coordinate, the learning dynamics differ depending on whether $\pi\approx 0$ (mostly taking the action leading to $s_2$) versus $\pi\approx 1$ (mostly taking the action leading to $s_1$). In other words, the update target involves $\pi\,v_1+(1-\pi)\,v_2$, which shifts the update direction; thus, $\pi=0$ and $\pi=1$ lead to different “pulls” in weight space. To be more detailed one can derive:

The one‑step lookahead (target) value under policy π is
$$
\bar v \;=\;\pi\,v_1 + (1-\pi)\,v_2,
$$
where
$$
v_1 = w^\top x_1 = w_1 + w_2,\quad
v_2 = w^\top x_2 = 2w_1 + w_2.
$$

Because all rewards are zero, the forward‑view TD(λ) target simplifies to
$$
G_\lambda \;=\;\frac{\gamma(1-\lambda)}{1-\gamma\lambda}\,\bar v \;=\;c(\lambda)\,\bigl((2-\pi)w_1 + w_2\bigr),
$$
with $c(\lambda)=\frac{\gamma(1-\lambda)}{1-\gamma\lambda}$.

The expected weight update under behaviour μ is
$$
\mathbb{E}[\Delta w]
=\alpha\Bigl[\mu\bigl(G_\lambda - v_1\bigr)x_1
+(1-\mu)\bigl(G_\lambda - v_2\bigr)x_2\Bigr].
$$

Substitute $x_1=[1,1]^\top$, $x_2=[2,1]^\top$, $v_1=w_1+w_2$, $v_2=2w_1+w_2$:
$$
\mathbb{E}[\Delta w]
=\alpha\begin{bmatrix}
\mu\bigl(c((2-\pi)w_1+w_2)-(w_1+w_2)\bigr)
+2(1-\mu)\bigl(c((2-\pi)w_1+w_2)-(2w_1+w_2)\bigr)\\[6pt]
\mu\bigl(c((2-\pi)w_1+w_2)-(w_1+w_2)\bigr)
+(1-\mu)\bigl(c((2-\pi)w_1+w_2)-(2w_1+w_2)\bigr)
\end{bmatrix}.
$$



 Compare π=0 versus π=1

1. **π=0** ⇒ $G_\lambda=c(2w_1+w_2)$.
   $$
   \mathbb{E}[\Delta w]_{\pi=0}
   =\alpha\begin{bmatrix}
   \mu\bigl((2c-1)w_1+(c-1)w_2\bigr)
   +2(1-\mu)\bigl((2c-2)w_1+(c-1)w_2\bigr)\\[4pt]
   \mu\bigl((2c-1)w_1+(c-1)w_2\bigr)
   +(1-\mu)\bigl((2c-2)w_1+(c-1)w_2\bigr)
   \end{bmatrix}.
   $$

2. **π=1** ⇒ $G_\lambda=c(w_1+w_2)$.
   $$
   \mathbb{E}[\Delta w]_{\pi=1}
   =\alpha\begin{bmatrix}
   \mu\bigl((c-1)(w_1+w_2)\bigr)
   +2(1-\mu)\bigl((c-2)w_1+(c-1)w_2\bigr)\\[4pt]
   \mu\bigl((c-1)(w_1+w_2)\bigr)
   +(1-\mu)\bigl((c-2)w_1+(c-1)w_2\bigr)
   \end{bmatrix}.
   $$

Since these two update vectors are **not scalar multiples** of each other (due to different coefficients on $w_1$ and $w_2$), the direction of the expected weight update depends on π, proving the observed asymmetry in the plots.

---
4. In linear function approximation with a forward-view TD($\lambda$) update, convergence occurs if the effective update matrix has eigenvalues of magnitude less than 1. For our 2-state/2-feature setup, one can analyze the corresponding 2×2 system matrix. A simplified criterion  is that the scalar $\kappa(\lambda) = \frac{\gamma(1-\lambda)}{1-\gamma\lambda}$ and the scaling induced by $\pi$ should keep the update matrix stable. Concretely, for $\lambda<1$ and $\gamma<1$, $\kappa<\infty$, but if $\kappa$ and $\pi$ yield certain combinations that push the updates too aggressively in weight space, divergence can occur. In practice, it often diverges when $\pi$ is near 0 (heavy use of $\mathbf{x}_2$) and $\lambda$ is high.

Proof:
Starting from the expected TD(λ) weight update (with μ=0.5):

$$
\Delta w
= \frac{\alpha}{2}\bigl[(G_\lambda - v_1)x_1 + (G_\lambda - v_2)x_2\bigr],
$$
where
$$
G_\lambda
= c(\lambda)\bigl(\pi v_1 + (1-\pi)v_2\bigr),
\quad
c(\lambda)=\frac{\gamma(1-\lambda)}{1-\gamma\lambda}.
$$

Writing this as a linear map \(\Delta w = \tfrac{\alpha}{2}X\,w\), one finds

$$
X =
\begin{pmatrix}
3c(\,2-\pi\,)-5 & 3c-3\\[4pt]
2c(\,2-\pi\,)-3 & 2c-2
\end{pmatrix}.
$$

Thus the one‑step update is
$$
w_{t+1} = w_t + \frac{\alpha}{2}X\,w_t
\quad\Longrightarrow\quad
w_{t+1} = A\,w_t,\quad
A = I + \frac{\alpha}{2}X.
$$

Stability via Jury Conditions

For both eigenvalues of \(A\) to lie strictly inside the unit circle, the following must hold:

- i $1 - \mathrm{tr}(A) + \det(A) > 0$
- ii $1 + \mathrm{tr}(A) + \det(A) > 0$
- iii $1 - \det(A) > 0$

With $\alpha=0.1$, direct calculation yields

- $\det(A)>0$ and condition(i) reduce to $c(\lambda)<1$, always true for $\lambda<1,\gamma<1$.
- Condition(iii) is also always satisfied.
- Condition(ii) simplifies exactly to

$$
\kappa(3\pi - 8) + 7 > 0,
\quad
\kappa = \frac{\gamma(1-\lambda)}{1-\gamma\lambda}.
$$

Rewriting gives the final convergence criterion:

$$
3\lambda\pi - 3\pi - \lambda \;<\; \frac{7}{\gamma} - 8.
$$
Whenever this inequality holds, the spectral radius of \(A\) is <1 and the expected update converges; otherwise it diverges.


---
5. In the reward-free case, the return $G_t$ is always zero, so the Monte Carlo update:
$$
\mathbb{E}[\Delta w]
=\alpha\Bigl[\mu\bigl(G_\lambda - v_1\bigr)x_1
+(1-\mu)\bigl(G_\lambda - v_2\bigr)x_2\Bigr].
$$
reduces to:
$$
\Delta \mathbf{w}\propto -v(s)\mathbf{x}(s)
$$
This update does not depend on $\pi$ at all (no 1-step lookahead, etc.). Consequently, all runs with $\lambda=1$ have identical updates—hence the same trajectories in weight space—regardless of $\pi$.
---
6. With $\lambda=1$, the update is $\Delta \mathbf{w}\approx -\alpha \bigl[\mu\,v(s_1)\,\mathbf{x}_1 + (1-\mu)\,v(s_2)\,\mathbf{x}_2\bigr]$. Because $\mathbf{x}_2$ has a larger first component than $\mathbf{x}_1$, there is a stronger pull on $w_1$ initially, causing $w_1$ to drop quickly. Once $w_1$ gets small or negative, the ratio of updates shifts, creating a characteristic bend in the trajectory as $w_2$ continues to adjust.
---
7. The expected update is

$$
\mathbb{E}_\mu[\Delta w]
= \alpha\Bigl[\mu\,x_1\,(G_\lambda - v_1) + (1-\mu)\,x_2\,(G_\lambda - v_2)\Bigr].
$$

Because $x_2=[2,1]^\top$ has a larger first‑coordinate than $x_1=[1,1]^\top$, updates from state $s_2$ pull $w_1$ twice as strongly (in the negative direction when $G_\lambda<v_2$) as updates from $s_1$. As $G_\lambda$ grows, $(G_\lambda-v_1)$ becomes positive before $(G_\lambda-v_2)$, so the $s_1$ term starts pushing $w$ away from zero sooner.

- **Higher $\mu$ (e.g. 0.8)** weights the $s_1$ update more heavily, lowering the threshold at which the update becomes positive along $w_1$, thus accelerating divergence.
- **Lower $\mu$ (e.g. 0.2)** gives more weight to $s_2$, whose stronger negative pull on $w_1$ delays the onset of positive updates and keeps weights moving toward zero longer, making convergence more likely.


